# Introduction
## Research Question 

What is the information flow from visual stream to motor processing and how early in processing can we predict behavioural outcomes.
- Can decoding models be trained by region 
- How accurate are the modeled regions at predicting a behaviour 
- Possible behaviours (correct vs. incorrect) 
- Movement of wheel

## Brief background
The Steinmetz (2018) dataset reported that neurons with action correlates are found globally and that neurons in nearly every brain region are non-selectively activated in the moments leading up to movement onset, however it is currently not known how the information integration occurs across the motor areas and how that integration gives rise to motor behaviour.

Neuron population coding has been robustly used to decode motor behaviours across various species (Georgopoulos et al., 1986), and recent literature has suggested that motor preparation and planning uses distributed populations in corticomotor areas to plan motor movements. However this previous work has been limited by the number of electrodes and therefore areas measured in a single task.

The following assignment seeks to take advantage of the multi-array recording from the Steinmetz (2018) neuropixel data set to investigate temporal aspects of motor behaviours.

# Data Analyses
:brain: :mouse: :brain:
## Set Up

In [2]:
import pandas as pd
import numpy as np
import dataframe_image as dfi
from matplotlib import rcParams 
from matplotlib import pyplot as plt

rcParams['figure.figsize'] = [15, 5]
rcParams['font.size'] = 15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import os, requests

import pathlib
pathlib.Path('/Users/sophiabatchelor/Code/Interview/SteinmetzData').mkdir(parents=True, exist_ok=True)

fname = []
for j in range(3):
    fname.append('steinmetz_part%d.npz'%j)
url = ['https://osf.io/agvxh/download']
url.append('https://osf.io/uv3mw/download')
url.append('https://osf.io/ehmw2/download')

for i in range(len(url)):
    if not os.path.isfile(fname[i]):
        try:
            r = requests.get(url[i])
        except requests.ConnectionError:
            print("Data could not download!")
        else:
            if r.status_code != requests.codes.ok:
                print("Data could not download!")
            else:
                with open(fname[i], "wb") as fid:
                    fid.write(r.content)
                    

steinmetz_data = np.array([])
for i in range(len(fname)):
    steinmetz_data = np.hstack((steinmetz_data, np.load('/Users/sophiabatchelor/Code/Interview/SteinmetzData'%i, allow_pickle=True)['dat']))

## View the dataset

In [ ]:
# choose one recording session (20) to get labels
session_20 = steinmetz_data[20]
keys = session_20.keys()
print(keys)

In [ ]:
for key in session_20.keys():
    dataset_info = session_20[key]
    if isinstance (dataset_info, np.ndarray):
        print(key, dataset_info.shape, " - array")
    elif isinstance (dataset_info, list):
        print(key, len(dataset_info), " - list")
    else: 
        print(key, type(dataset_info), " - other")

In [ ]:
table = pd.DataFrame(columns=['session', 'mouse_name', 'n_neuron'] + ubs)

for i in range(steinmetz_data.shape[0]):
    this_session: dict = {}
    unique_area = list(np.unique(steinmetz_data[i]['brain_area']))
    this_session['session'] = i
    this_session['mouse_name'] = steinmetz_data[i]['mouse_name']
    this_session['n_neuron'] = steinmetz_data[i]['spks'].shape[0]
    this_session['n_trial'] = steinmetz_data[i]['spks'].shape[1]
    
    for ea_new_area in unique_area:
        n_neuron, n_trial, _ = (steinmetz_data[i]['spks'][steinmetz_data[i]['brain_area'] == ea_new_area]).shape
        this_session[ea_new_area] = n_neuron

    table = table.append(this_session, ignore_index=True)
table = table.fillna(0)
pathlib.Path('/Users/sophiabatchelor/Code/SteinmetzAnalyses/Images').mkdir(parents=True, exist_ok=True)
dfi.export(table, '/Users/sophiabatchelor/Code/SteinmetzAnalyses/Images/steinmetz_all_data_table.png', max_cols=77)

table

## Data Wrangling

In [ ]:
names = []
for dat_i in steinmetz_data:
    name = dat_i["mouse_name"]
    if name not in names:
        names.append(name)
print("Mice: {}".format(names))

In [ ]:
brain_areas_by_session_list = []

for i in range(len(steinmetz_data)):
    data = steinmetz_data[i]
    session = i 
    brain_areas = data["brain_area"]
    areas_by_session = np.unique(brain_areas)
    print(areas_by_session)
    brain_areas_by_session_list.append(areas_by_session)
#     that_sessions_go_cue_avg = np.mean(data['gocue']) TODO: decide on which gocou avg is needed
#     print(i)
#     print(data["mouse_name"])
#     print(that_sessions_go_cue_avg)
    

In [ ]:
brain_areas = []
for i in range(steinmetz_data.shape[0]):
    unique_area = np.unique(steinmetz_data[i]['brain_area']) # check this line for the 
    for u in unique_area:
        brain_areas.append(u)
ubs = list(np.unique(brain_areas))

In [3]:
mouse_dict = {} # create a dictionary 

for session, dat_i in enumerate(steinmetz_data):
    name = dat_i["mouse_name"]
    if name not in mouse_dict.keys():
        mouse_dict[name] = [dat_i]

    else:
        lst = mouse_dict[name]
        lst.append(dat_i)
        mouse_dict[name] = lst

NameError: name 'steinmetz_data' is not defined

In [ ]:
for assigned_region in assigned_regions:
    all_mice_names = []
    all_mice_lines = None

    for mouse in mouse_dict.keys():
        mouse_data = mouse_dict[mouse] 
        num_sessions = len(mouse_dict[mouse])

        spk_all_sessions = None
        
        for session in mouse_data:
            spk_session = session['spks']            
            if assigned_region in session['brain_area']:
                spk_session_region = spk_session[session['brain_area'] == assigned_region]

                # average over trials
                spk_session_region_avg = np.mean(spk_session_region, axis=1)

                if spk_all_sessions is not None:
                    spk_all_sessions = np.concatenate((spk_all_sessions, spk_session_region_avg))
                else:
                    spk_all_sessions = spk_session_region_avg
                    

        # average over all neurons
        if spk_all_sessions is not None:
            name_i = mouse
            all_mice_names.append(name_i)
            mouse_i = np.mean(spk_all_sessions, axis=0)
            mouse_i = np.expand_dims(mouse_i, 0)

            if all_mice_lines is not None:
                all_mice_lines = np.concatenate((all_mice_lines, mouse_i), axis = 0)
            else:
                all_mice_lines = mouse_i 

    plot = plt.figure(figsize=(20, 10))
    plt.plot(all_mice_lines.T) # had to transpose so that time was on the x axis 

    plot.suptitle("Average Spiking of {}".format(assigned_region))
    plt.xlabel("Timebins")  # change axis labels if you need reminders
    plt.ylabel("Average Number of Spikes per time bin")
    plt.legend(all_mice_names, loc = "upper right")
    pathlib.Path('/Users/sophiabatchelor/Code/SteinmetzAnalyses/Images/Plots').mkdir(parents=True, exist_ok=True)
    plt.savefig('/Users/sophiabatchelor/Code/SteinmetzAnalyses/Images/Plots/Plotof{}.png'.format(assigned_region))
    plt.show()

In [ ]:
## NEXT STEPS:

# output the plots using subplot 
# get the axes aligned
# and also plot the go_cues with them 